In [1]:
#%% import
import os
from glob import glob
import re
import ast
import numpy as np 
import pandas as pd
from PIL import Image, ImageDraw 
from tqdm import tqdm
from dask import bag

import tensorflow as tf
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.metrics import top_k_categorical_accuracy
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
classfiles = os.listdir('./train_simplified/')
numstonames = {i: v[:-4].replace(" ", "_") for i, v in enumerate(classfiles)}

num_classes = 340
imheight, imwidth = 32, 32
ims_per_class = 2000

In [3]:
# faster conversion function
def draw_it(strokes):
    image = Image.new("P", (256,256), color=255)
    image_draw = ImageDraw.Draw(image)
    for stroke in ast.literal_eval(strokes):
        for i in range(len(stroke[0])-1):
            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=0, width=5)
    image = image.resize((imheight, imwidth))
    return np.array(image)/255.

train_grand = []
class_paths = glob('./train_simplified/*.csv')
for i,c in enumerate(tqdm(class_paths[0:num_classes])):
# for i,c in enumerate(tqdm(class_paths[0:2])):
    train = pd.read_csv(c, usecols=['drawing', 'recognized'], nrows=ims_per_class*5//4)
    train = train[train.recognized == True].head(ims_per_class)
    imagebag = bag.from_sequence(train.drawing.values).map(draw_it)
    trainarray = np.array(imagebag.compute())
    trainarray = np.reshape(trainarray, (ims_per_class, -1))
    labelarray = np.full((train.shape[0], 1), i)
    trainarray = np.concatenate((labelarray, trainarray), axis=1)
    train_grand.append(trainarray)

# print(type(train_grand))
train_grand = np.array([train_grand.pop() for i in np.arange(num_classes)])
# print(type(train_grand))
train_grand = train_grand.reshape((-1, (imheight*imwidth+1)))
# print(type(train_grand))

del trainarray
del train

100%|██████████| 340/340 [05:09<00:00,  1.10it/s]


In [4]:
valfrac = 0.1

# print(train_grand.shape[0])
cutpt = int(valfrac * train_grand.shape[0])
# print(cutpt)

np.random.shuffle(train_grand)
y_train, X_train = train_grand[cutpt: , 0], train_grand[cutpt:, 1:]
print(y_train.shape)
print(X_train.shape)
y_val, X_val = train_grand[0:cutpt, 0], train_grand[0:cutpt, 1:]
print(y_val.shape)
print(X_val.shape)

del train_grand

y_train = keras.utils.to_categorical(y_train, num_classes)
X_train = X_train.reshape(X_train.shape[0], imheight, imwidth, 1)
y_val = keras.utils.to_categorical(y_val, num_classes)
X_val = X_val.reshape(X_val.shape[0], imheight, imwidth, 1)

print(y_train.shape, "\n",
      X_train.shape, "\n",
      y_val.shape, "\n",
      X_val.shape)

(612000,)
(612000, 1024)
(68000,)
(68000, 1024)
(612000, 340) 
 (612000, 32, 32, 1) 
 (68000, 340) 
 (68000, 32, 32, 1)


In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(imheight, imwidth, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(680, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 680)               2785960   
__________

In [6]:
def top_3_accuracy(x, y):
    t3 = top_k_categorical_accuracy(x, y, 3)
    return t3

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
                                   verbose=1, mode='auto', cooldown=5, min_lr=0.0001)

earlystop = EarlyStopping(monitor='val_top_3_accuracy', mode='max', patience=5)
callbacks = [reduceLROnPlat, earlystop]

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy', top_3_accuracy])

model.fit(x=X_train, y=y_train, batch_size=32, epochs=22, 
          validation_data=(X_val, y_val), callbacks=callbacks, verbose=1)


Train on 612000 samples, validate on 68000 samples
Epoch 1/22
612000/612000 [==============================] - 94s 154us/step - loss: 3.3640 - acc: 0.2726 - top_3_accuracy: 0.4455 - val_loss: 2.2731 - val_acc: 0.4548 - val_top_3_accuracy: 0.6641
Epoch 2/22
612000/612000 [==============================] - 91s 148us/step - loss: 2.5620 - acc: 0.3924 - top_3_accuracy: 0.6033 - val_loss: 2.0749 - val_acc: 0.4950 - val_top_3_accuracy: 0.7058
Epoch 3/22
612000/612000 [==============================] - 91s 149us/step - loss: 2.4146 - acc: 0.4194 - top_3_accuracy: 0.6332 - val_loss: 1.9708 - val_acc: 0.5120 - val_top_3_accuracy: 0.7244
Epoch 4/22
612000/612000 [==============================] - 91s 149us/step - loss: 2.3418 - acc: 0.4347 - top_3_accuracy: 0.6487 - val_loss: 1.9401 - val_acc: 0.5215 - val_top_3_accuracy: 0.7335
Epoch 5/22
612000/612000 [==============================] - 92s 150us/step - loss: 2.3012 - acc: 0.4424 - top_3_accuracy: 0.6567 - val_loss: 1.9305 - val_acc: 0.5244 - v

In [7]:
ttvlist = []
reader = pd.read_csv('./test_simplified.csv', index_col=['key_id'],
    chunksize=2048)
for chunk in tqdm(reader, total=55):
    imagebag = bag.from_sequence(chunk.drawing.values).map(draw_it)
    testarray = np.array(imagebag.compute())
    testarray = np.reshape(testarray, (testarray.shape[0], imheight, imwidth, 1))
    testpreds = model.predict(testarray, verbose=0)
    ttvs = np.argsort(-testpreds)[:, 0:3]  # top 3
    ttvlist.append(ttvs)

ttvarray = np.concatenate(ttvlist)

100%|██████████| 55/55 [01:25<00:00,  1.56s/it]


In [8]:
preds_df = pd.DataFrame({'first': ttvarray[:,0], 'second': ttvarray[:,1], 'third': ttvarray[:,2]})
preds_df = preds_df.replace(numstonames)
preds_df['words'] = preds_df['first'] + " " + preds_df['second'] + " " + preds_df['third']

sub = pd.read_csv('./sample_submission.csv', index_col=['key_id'])
sub['word'] = preds_df.words.values
sub.to_csv('subcnn_small.csv')
sub.head()

,word
key_id,
9000003627287624,nose remote_control peas
9000010688666847,crocodile hot_tub bulldozer
9000023642890129,lollipop telephone pond
9000038588854897,ceiling_fan face stethoscope
9000052667981386,bowtie firetruck sea_turtle


In [10]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not 
    x.startswith('_') and x not in sys.modules and x 
    not in ipython_vars], key=lambda x: x[1], reverse=True)

[('preds_df', 30114166),
 ('sub', 9671503),
 ('ttvarray', 2692888),
 ('chunk', 1053432),
 ('testpreds', 218664),
 ('labelarray', 16112),
 ('numstonames', 9320),
 ('class_paths', 3104),
 ('Sequential', 3096),
 ('classfiles', 2896),
 ('Dense', 2000),
 ('Dropout', 2000),
 ('Flatten', 2000),
 ('MaxPooling2D', 2000),
 ('EarlyStopping', 1464),
 ('ReduceLROnPlateau', 1464),
 ('ModelCheckpoint', 1056),
 ('Conv2D', 888),
 ('ttvlist', 528),
 ('X_train', 144),
 ('X_val', 144),
 ('testarray', 144),
 ('draw_it', 136),
 ('top_3_accuracy', 136),
 ('top_k_categorical_accuracy', 136),
 ('ttvs', 112),
 ('y_train', 112),
 ('y_val', 112),
 ('c', 81),
 ('Image', 80),
 ('ImageDraw', 80),
 ('bag', 80),
 ('callbacks', 80),
 ('keras', 80),
 ('np', 80),
 ('pd', 80),
 ('tf', 80),
 ('earlystop', 56),
 ('imagebag', 56),
 ('model', 56),
 ('reader', 56),
 ('reduceLROnPlat', 56),
 ('cutpt', 28),
 ('i', 28),
 ('imheight', 28),
 ('ims_per_class', 28),
 ('imwidth', 28),
 ('num_classes', 28),
 ('valfrac', 24)]